## 1. LLM Chain

In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history"
)

In [4]:
template = """
    You are a helpful AI talking to a human.

    {chat_history}
    Human: {question}
    You:
"""

In [5]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template(template),
    verbose=True
)

In [6]:
chain.predict(question="Hi. My name is Ong")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    
    Human: Hi. My name is Ong
    You:


> Finished chain.


'Hello Ong! How can I assist you today?'

In [7]:
chain.predict(question="I live in Dongtan in Korea.")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: Hi. My name is Ong
AI: Hello Ong! How can I assist you today?
    Human: I live in Dongtan in Korea.
    You:


> Finished chain.


"That's great to know! How can I assist you with anything related to Dongtan or Korea?"

In [9]:
chain.predict(question="What do you know about me?")



> Entering new LLMChain chain...
Prompt after formatting:

    You are a helpful AI talking to a human.

    Human: Hi. My name is Ong
AI: Hello Ong! How can I assist you today?
Human: I live in Dongtan in Korea.
AI: That's great to know! How can I assist you with anything related to Dongtan or Korea?
    Human: What do you know about me?
    You:


> Finished chain.


'I know that your name is Ong and you live in Dongtan, Korea. Is there anything specific you would like to know or discuss?'

## 2. Chat Based Memory

In [8]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [10]:
llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="history",
    return_messages=True
)

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}")
    ]
)

In [12]:
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True
)

In [13]:
chain.predict(question="My name is Ong.")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Ong.

> Finished chain.


'Nice to meet you, Ong! How can I assist you today?'

In [14]:
chain.predict(question="I live in Dongtan in Korea")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
Human: My name is Ong.
AI: Nice to meet you, Ong! How can I assist you today?
Human: I live in Dongtan in Korea

> Finished chain.


"That's great to know, Ong! Is there anything specific you would like to know or talk about regarding Dongtan or Korea?"

In [16]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human.
System: The human introduces themselves as Ong. The AI responds by saying, "Nice to meet you, Ong! How can I assist you today?"
Human: I live in Dongtan in Korea
AI: That's great to know, Ong! Is there anything specific you would like to know or talk about regarding Dongtan or Korea?
Human: What do you know about me?
AI: As an AI, I don't have access to personal information unless you provide it during our conversation. I'm here to assist you with any questions or information you may need. How can I help you today, Ong?
Human: What is my name?

> Finished chain.


'Your name is Ong, as you introduced yourself earlier in our conversation. How can I assist you further, Ong?'

## 3. LCEM Based Memory

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

In [2]:

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

In [3]:
def load_memory(input):
    return memory.load_memory_variables({})["history"]

In [4]:
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

In [5]:
def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [6]:
invoke_chain("My name is Ong.")

content='Nice to meet you, Ong! How can I assist you today?'


In [7]:
invoke_chain("What is my name?")

content='Your name is Ong.'
